In [224]:
from models.yolo import Detect
from utils.general import scale_coords, non_max_suppression, xyxy2xywh, xyxy2xywhn
import coremltools
from PIL import Image
import torch
import numpy as np
import random
import cv2
import os
import shutil
import PIL.Image

In [221]:
# CONSTANTS
# COREML_MODEL = "/Users/zhenyu/Documents/Scripts/IphoneAOI/yolov5/best_1106.mlmodel"
COREML_MODEL = "/Users/zhenyu/Desktop/exp6/weights/best.mlmodel"
IMAGE_FOLDER = "/Users/zhenyu/Desktop/val/"
OUT_FOLDER = "/Users/zhenyu/Desktop/test/"
SAVE_IMG = True
VIEW_IMG = False
SAVE_TXT = True
CAT_NAMES = ['Screw', 'unknown']
COLORS = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(CAT_NAMES))]
PATH = "./"
ANCHORS = ([116,90, 156,198, 373,326], [30,61, 62,45, 59,119], [10,13, 16,30, 33,23]) # from <model>.yml
IMG_SIZE = (2304, 3072)
# IMG_SIZE = (2560, 2560)

# GLOBAL VARIABLES
nc = len(CAT_NAMES)
nl = len(ANCHORS)
na = len(ANCHORS[0]) // 2
no = nc + 5  # number of outputs per anchor
grid = [torch.zeros(1)] * nl  # init grid
a = torch.tensor(ANCHORS).float().view(nl, -1, 2)
anchor_grid = a.clone().view(nl, 1, -1, 1, 1, 2)
# stride for 2304,3072
stride = [32, 8, 16]
# stride for 2560,2560
# stride = [32, 64, 16]
conf_thres = .3

In [191]:
def load_image(path, resize_to=None):
    # resize_to: (Width, Height)
    img = PIL.Image.open(path)
    if resize_to is not None:
        img = img.resize(resize_to, PIL.Image.ANTIALIAS)
    img_np = np.array(img).astype(np.float32)
    return img_np, img

def make_grid(nx=20, ny=20):
    yv, xv = torch.meshgrid([torch.arange(ny), torch.arange(nx)])
    return torch.stack((xv, yv), 2).view((1, 1, ny, nx, 2)).float()

def resize_image(source_image):
    background = PIL.Image.new('RGB', IMG_SIZE, "black")
    source = source_image.copy()
    source.thumbnail(IMG_SIZE)
    (w, h) = source.size
    background.paste(source, (int((IMG_SIZE[0] - w) / 2), int((IMG_SIZE[1] - h) / 2 )))
    return background

In [40]:
source = PIL.Image.open(os.path.join(IMAGE_FOLDER, '2.jpg'))
resized = resize_image(source)

In [41]:
source.size

(3024, 4032)

In [292]:
def eval(file_name):   
    source = PIL.Image.open(os.path.join(IMAGE_FOLDER, file_name))
    resized = resize_image(source)

    predictions = model.predict({'image': resized})

    z = []  # inference output
    x = []
    for pred in predictions:
        x.append(torch.Tensor(predictions[pred]))
    x.reverse()

    for i in range(nl):
        bs, _, ny, nx, _ = x[i].shape

        if grid[i].shape[2:4] != x[i].shape[2:4]:
            grid[i] = make_grid(nx, ny)

        y = x[i].sigmoid()
        y[..., 0:2] = (y[..., 0:2] * 2. - 0.5 + grid[i]) * stride[i]  # xy
        y[..., 2:4] = (y[..., 2:4] * 2) ** 2 * anchor_grid[i]  # wh
        z.append(y.view(bs, -1, no))
    
    pred = torch.cat(z, 1)

    pred = non_max_suppression(pred, conf_thres, .3, classes=None, agnostic=False)

    # Process detections
    for i, det in enumerate(pred):  # detections per image
        p, s = "./", ""
        label = []

        if det is not None and len(det):
            # Rescale boxes from img_size to im0 size
            # det = det[((det[:, 0]-det[:, 2])*(det[:, 1]-det[:, 3])) > 80]

            # Print results
#             for c in det[:, -1].unique():
#                 n = (det[:, -1] == c).sum()  # detections per class
#                 s += '%g %ss, ' % (n, CAT_NAMES[int(c)])  # add to string

            # Write results
            for *xyxy, conf, cls in det:
                if SAVE_TXT:
                    xywh = xyxy2xywhn(torch.tensor(xyxy).view(1, 4), w=IMG_SIZE[0], h=IMG_SIZE[1]).view(-1).tolist()
                    label.append(('%g ' * 5 + '\n') % (cls, *xywh))
                if SAVE_IMG:
                    draw = PIL.ImageDraw.Draw(source)
                    draw.rectangle(np.array(torch.tensor(xyxy).view(2,2)*1.3125), outline='red')
        if SAVE_TXT:
            with open(os.path.join(OUT_FOLDER, '{}.txt'.format(file_name[:-4])), 'a') as f:
                for line in label:
                    f.write(line)
        if SAVE_IMG:
            source.save(os.path.join(OUT_FOLDER, '{}.jpg'.format(file_name[:-4])))

In [193]:
model = coremltools.models.model.MLModel(COREML_MODEL)

In [293]:
eval('P9YI6XTHXQ.jpg')

In [287]:
type(x[0])

str

In [291]:
with open(os.path.join(OUT_FOLDER, 'dda.txt'), 'a') as f:
    for line in x:
        f.write(line)

In [152]:
for each in predictions.keys():
    print('{}:{}'.format(each, predictions[each].shape))

var_1808:(1, 3, 48, 36, 7)
var_1778:(1, 3, 192, 144, 7)
var_1763:(1, 3, 384, 288, 7)
var_1793:(1, 3, 96, 72, 7)


In [213]:
y = pred[0][((pred[0][:, 0]-pred[0][:, 2])*(pred[0][:, 1]-pred[0][:, 3])) > 80]

In [214]:
y

tensor([[1.45282e+03, 1.45009e+03, 1.50923e+03, 1.53559e+03, 8.58536e-01, 0.00000e+00],
        [8.11149e+02, 2.51468e+02, 8.65843e+02, 3.34357e+02, 8.56946e-01, 0.00000e+00],
        [1.32890e+03, 6.14181e+02, 1.38540e+03, 6.95824e+02, 8.49743e-01, 0.00000e+00],
        [1.32848e+03, 1.39324e+03, 1.36927e+03, 1.45567e+03, 8.44309e-01, 0.00000e+00],
        [8.95427e+02, 1.55035e+02, 9.40290e+02, 2.22714e+02, 8.43017e-01, 0.00000e+00],
        [8.04034e+02, 4.29946e+02, 8.54911e+02, 5.09081e+02, 8.40511e-01, 0.00000e+00],
        [1.42461e+03, 2.20768e+03, 1.47910e+03, 2.29305e+03, 8.38835e-01, 0.00000e+00],
        [1.33620e+03, 1.30665e+03, 1.38581e+03, 1.38278e+03, 8.38572e-01, 0.00000e+00],
        [7.46007e+02, 1.63791e+02, 7.79548e+02, 2.19061e+02, 8.37310e-01, 0.00000e+00],
        [1.14234e+03, 7.25872e+02, 1.18160e+03, 7.85555e+02, 8.33452e-01, 0.00000e+00],
        [1.28350e+03, 1.69889e+03, 1.34053e+03, 1.78646e+03, 8.31392e-01, 0.00000e+00],
        [1.20805e+03, 1.23356e+0

In [212]:
pred

[tensor([[1.45282e+03, 1.45009e+03, 1.50923e+03, 1.53559e+03, 8.58536e-01, 0.00000e+00],
         [8.11149e+02, 2.51468e+02, 8.65843e+02, 3.34357e+02, 8.56946e-01, 0.00000e+00],
         [1.32890e+03, 6.14181e+02, 1.38540e+03, 6.95824e+02, 8.49743e-01, 0.00000e+00],
         [1.32848e+03, 1.39324e+03, 1.36927e+03, 1.45567e+03, 8.44309e-01, 0.00000e+00],
         [8.95427e+02, 1.55035e+02, 9.40290e+02, 2.22714e+02, 8.43017e-01, 0.00000e+00],
         [8.04034e+02, 4.29946e+02, 8.54911e+02, 5.09081e+02, 8.40511e-01, 0.00000e+00],
         [1.42461e+03, 2.20768e+03, 1.47910e+03, 2.29305e+03, 8.38835e-01, 0.00000e+00],
         [1.33620e+03, 1.30665e+03, 1.38581e+03, 1.38278e+03, 8.38572e-01, 0.00000e+00],
         [7.46007e+02, 1.63791e+02, 7.79548e+02, 2.19061e+02, 8.37310e-01, 0.00000e+00],
         [1.14234e+03, 7.25872e+02, 1.18160e+03, 7.85555e+02, 8.33452e-01, 0.00000e+00],
         [1.28350e+03, 1.69889e+03, 1.34053e+03, 1.78646e+03, 8.31392e-01, 0.00000e+00],
         [1.20805e+03